In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Reinforcement Learning
from src.reinforcement_learning.env import HyperHeuristicEnv 

# Hyperheuristic
from src.HyperHeuristic import HyperHeuristic

from src.initial_solution.ConstructiveHeuristic import ConstructiveHeuristic, RandomConstructive, GreedyRandomizedConstructive, GreedyConstructive
from src.operators.DestroyOperators import RandomRemove, WorstRemove 
from src.operators.RepairOperators import RandomRepair, GreedyRepair
from src.local_search.LocalSearch import FirstImprovement

from src.accept.SimulatedAnnealing import SimulatedAnnealing
from src.accept.RecordToRecordTravel import RecordToRecordTravel

from src.stop.MaxRuntimeOrNoImprovement import MaxRuntimeOrNoImprovement

# Instances
from src.KnapsackInstance import KnapsackInstance
from utils.benchmark import BinaryKnapsackBenchmark
from utils.knapsackSortFunctions import sortIndexesByProfitWeightDensity

# Misc libraries
import pandas as pd
import numpy as np
import random
import re

import plotly.express as px

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

# **Hyperheuristic Construction**

In [3]:
def on_best(cand, rand):
    # Write your code here
    return cand

In [4]:
stop = MaxRuntimeOrNoImprovement(max_runtime = 60, max_iterations = 1000)

recordToRecordTravel = RecordToRecordTravel(
    start_threshold = 0.5,
    end_threshold = 0.1,
    step = 0.01,
    method = 'linear',
    cmp_best = True
)

# simulatedAnnealingCriteria = SimulatedAnnealing(
#     start_temperature = 100,
#     end_temperature = 0.001,
#     step = 0.99,
#     method = 'exponential'
# )

randomRemove1 = RandomRemove(0.2)
randomRemove2 = RandomRemove(0.4)
worstRemove1 = WorstRemove(0.2)
worstRemove2 = WorstRemove(0.4)
randomRepair = RandomRepair()
greedyRepair = GreedyRepair()

firstImprovement = FirstImprovement()

randomConstruct = RandomConstructive()
greedyConstruct = GreedyConstructive(sortIndexesByProfitWeightDensity)
greedyRandomizedConstruct = GreedyRandomizedConstructive(0.3)

rewards = [5,3,1,0.5]

In [5]:
hyperHeuristic = HyperHeuristic(stop = stop, acceptance = recordToRecordTravel, rewards = rewards, on_best = on_best)

hyperHeuristic.add_refinement(randomRemove1, 'randomRemove0.2')
hyperHeuristic.add_refinement(randomRemove2, 'randomRemove0.4')
hyperHeuristic.add_refinement(worstRemove1, 'worstRemove0.2')
hyperHeuristic.add_refinement(worstRemove2, 'worstRemove0.4')
hyperHeuristic.add_refinement(randomRepair, 'randomRepair')
hyperHeuristic.add_refinement(greedyRepair, 'greedyRepair')

hyperHeuristic.add_refinement(firstImprovement, 'firstImprovement')

hyperHeuristic.add_refinement(randomConstruct, 'randomConstruct')
hyperHeuristic.add_refinement(greedyConstruct, 'greedyConstruct')
hyperHeuristic.add_refinement(greedyRandomizedConstruct, 'greedyRandomizedConstruct')

# **Benchmark**

In [6]:
benchmarkPath = "../data/benchmark/binaryKnapsack"
benchmark = BinaryKnapsackBenchmark("binary", benchmarkPath)
instancesList = benchmark.getInstancesList()
np.random.shuffle(instancesList)

# **Q-Learning**

In [7]:
def instanceGenerator(instancesNames):
    while True:
        instanceName = np.random.choice(instancesNames, 1, replace = True)[0]
        instance = benchmark.parseInstance(instanceName)
        yield (instanceName, instance)

In [8]:
train, test = train_test_split(instancesList, test_size = 0.2)
trainInstancesGenerator = instanceGenerator(train)

In [9]:
env = HyperHeuristicEnv(hyperHeuristic, trainInstancesGenerator)

In [14]:
from IPython.display import clear_output

q_tables = []
q_table = np.zeros([env.observation_space.n, env.action_space.n])
q_tables.append((0, np.array(q_table, dtype = np.double)))

# Hyperparameters
alpha_start = 0.5
alpha_end = 0.05
gamma = 0.6
epsilon = 0.1
n_epochs = 15000

# For plotting metrics
all_epochs = []

for i in range(1, n_epochs + 1):
    state = env.reset()

    totalEpochIterations, totalEpochReward, = 1, 0
    done = False
    
    while not done:
        alpha = alpha_start + (i-1) * (alpha_end - alpha_start)/n_epochs
        
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        totalEpochReward += reward
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        state = next_state
        totalEpochIterations += 1
        
    runningTime = env.get_running_time()

    all_epochs.append([totalEpochIterations, totalEpochReward, runningTime])

    if i % 250 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        # print(q_table)
        q_tables.append((i, np.array(q_table, dtype = np.double)))

print("Training finished.\n")

Episode: 15000
Training finished.



In [16]:
trainingSummary = pd.DataFrame(data = all_epochs, columns = ['iterations', 'reward', 'runnnigTime'])
trainingSummary.head()

,iterations,reward,runnnigTime
0,2,5.0,0.010700
1,2,0.5,0.018105
2,2,1.0,0.014024
3,2,5.0,0.017026
4,2,0.5,0.015514


## **Evaluation**

In [ ]:
columns = ['instanceName', 'optimumFO', 'bestFO', 'optimumRuntime', 'runtime', 'relativeError']
values = []

for instanceName in test:
    print(instanceName)
    instance = benchmark.parseInstance(instanceName)

    state = env.reset(instance)

    done = False

    while not done:
        action = np.argmax(q_tables[-1][1][state])
        state, reward, done, info = env.step(action)

    optimum = benchmark.getOptimalFO(instanceName)
    objective = env._hyperHeuristic.best_solution.objective(isMinimizing = False)
    runtime = env.get_running_time()
    optimumRuntime = benchmark.getOptimalRunningTime(instanceName)

    values.append([
        instanceName.replace('/test.in',''),
        optimum,
        objective,
        optimumRuntime,
        runtime,
        (optimum - objective)/optimum,
    ])

In [123]:
testSummary = pd.DataFrame(data = values, columns = columns)
testSummary.head()

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError
0,n_1200_c_100000000_g_6_f_0.1_eps_0.01_s_100,99756399,99007997,10.250944,0.007487,7.502296e-03
1,n_800_c_10000000000_g_2_f_0.2_eps_0.001_s_100,5010008771,5010008769,0.352251,0.005242,3.992009e-10
2,n_1200_c_100000000_g_6_f_0.3_eps_1e-05_s_200,96947654,96944653,15.953581,0.005845,3.095485e-05
3,n_1200_c_100000000_g_2_f_0.1_eps_0.001_s_100,50106076,50106073,0.196778,0.003467,5.987298e-08
4,n_1000_c_10000000000_g_2_f_0.2_eps_0.01_s_200,5100020242,5100020241,0.447970,0.004287,1.960777e-10


In [125]:
testSummary['n'] = testSummary.instanceName.apply(lambda name: re.findall(r"n_(\d+).*", name)[0])
testSummary['c'] = testSummary.instanceName.apply(lambda name: re.findall(r".*c_(\d+).*", name)[0])
testSummary.head()

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError,n,c
0,n_1200_c_100000000_g_6_f_0.1_eps_0.01_s_100,99756399,99007997,10.250944,0.007487,7.502296e-03,1200,100000000
1,n_800_c_10000000000_g_2_f_0.2_eps_0.001_s_100,5010008771,5010008769,0.352251,0.005242,3.992009e-10,800,10000000000
2,n_1200_c_100000000_g_6_f_0.3_eps_1e-05_s_200,96947654,96944653,15.953581,0.005845,3.095485e-05,1200,100000000
3,n_1200_c_100000000_g_2_f_0.1_eps_0.001_s_100,50106076,50106073,0.196778,0.003467,5.987298e-08,1200,100000000
4,n_1000_c_10000000000_g_2_f_0.2_eps_0.01_s_200,5100020242,5100020241,0.447970,0.004287,1.960777e-10,1000,10000000000


# **Visualization**

## **Train**

### **Q-Table convergence**

In [122]:
QAbsErrors = [(q_tables[i][0], abs(q_tables[i][1] - q_tables[i-1][1])) for i in range(1, len(q_tables))]
QErrors = [(errorTable[0], errorTable[1].mean(), errorTable[1].std()) for errorTable in QAbsErrors]
QErrorsDF = pd.DataFrame(data = QErrors, columns = ['epoch', 'mean', 'std'])
QErrorsDF.head()

,epoch,mean,std
0,250,1.526457,2.078199
1,500,0.194963,0.658449
2,750,0.183244,0.712249
3,1000,0.095744,0.387181
4,1250,0.106585,0.379360


In [121]:
fig = px.line(QErrorsDF, x = 'epoch', y = 'mean', title = 'Q-Table Convergence')

fig.update_layout(
    yaxis_title="Q-Table Mean Absulute Error",
    xaxis_title="episode"
)

## **Test**

In the cell below, we dropped all instances without a optimal solution and those instances for which we found an unfeasible solution. 

In [67]:
testStats = testSummary[(testSummary.optimumFO != -1) & (testSummary.bestFO >= 0)].drop(['instanceName', 'optimumFO', 'bestFO'], axis = 1).groupby(['n','c']).describe()
testStats

optimumRuntime                                               \
                          count        mean          std       min       25%   
n    c                                                                         
1000 1000000               40.0    0.695879     0.871804  0.066365  0.144612   
     100000000             43.0   36.674125    91.413375  0.081063  0.347560   
     10000000000           39.0  173.635860   703.052507  0.104318  0.390550   
1200 1000000               52.0    0.923247     1.334588  0.043045  0.113109   
     100000000             37.0   31.684580    67.135760  0.100219  0.802566   
     10000000000           30.0  390.303080  1064.718819  0.109388  0.502895   
400  1000000               41.0    0.209334     0.190658  0.046808  0.086646   
     100000000             31.0    4.673597    10.387513  0.079274  0.194732   
     10000000000           32.0   85.628287   159.758827  0.069193  0.156634   
600  1000000               50.0    0.394153     0.399047  0.037570  0.121023   
     100000000             39.0   24.248946    62.524312  0.064997  0.814956   
     10000000000           34.0   38.291434    86.219399  0.096465  0.300990   
800  1000000               48.0    0.342344     0.468991  0.044553  0.097801   
     100000000             46.0   16.027745    24.508058  0.124607  0.526738   
     10000000000           31.0  238.522235   622.223433  0.108661  0.303147   

                                                     runtime            ...  \
                        50%         75%          max   count      mean  ...   
n    c                                                                  ...   
1000 1000000       0.262752    0.911493     3.064962    40.0  0.005108  ...   
     100000000     5.313347   17.871624   502.545187    43.0  0.005577  ...   
     10000000000   2.864677   42.120280  4247.350154    39.0  0.005117  ...   
1200 1000000       0.291056    1.033663     5.382550    52.0  0.005898  ...   
     100000000     5.967343   17.986230   321.547525    37.0  0.006455  ...   
     10000000000  19.992735  118.397582  4795.614831    30.0  0.005683  ...   
400  1000000       0.114838    0.302980     0.966433    41.0  0.002411  ...   
     100000000     0.349379    3.141787    54.207020    31.0  0.002520  ...   
     10000000000   1.615674   70.223674   655.978393    32.0  0.002497  ...   
600  1000000       0.199370    0.521474     1.968798    50.0  0.003367  ...   
     100000000     2.864749   13.158541   336.886346    39.0  0.003592  ...   
     10000000000   3.169713   13.500229   298.392450    34.0  0.003421  ...   
800  1000000       0.146556    0.305196     2.100675    48.0  0.004178  ...   
     100000000     5.841022   20.520860    97.811773    46.0  0.004325  ...   
     10000000000   3.002987   50.097932  2553.091676    31.0  0.004952  ...   

                                     relativeError                      \
                       75%       max         count      mean       std   
n    c                                                                   
1000 1000000      0.005702  0.009633          40.0  0.007274  0.018743   
     100000000    0.006158  0.011637          43.0  0.005022  0.012870   
     10000000000  0.006108  0.009185          39.0  0.006437  0.016003   
1200 1000000      0.006381  0.013247          52.0  0.009792  0.021350   
     100000000    0.007032  0.016675          37.0  0.005213  0.015506   
     10000000000  0.006524  0.007949          30.0  0.013352  0.025306   
400  1000000      0.002843  0.004869          41.0  0.006108  0.014936   
     100000000    0.002936  0.004262          31.0  0.007207  0.016846   
     10000000000  0.002896  0.004301          32.0  0.008760  0.017160   
600  1000000      0.003774  0.007640          50.0  0.007920  0.019324   
     100000000    0.003883  0.009090          39.0  0.009350  0.020169   
     10000000000  0.003485  0.007809          34.0  0.008897  0.016723   
800  1000000      0.004966  0.006519       

Next, we focus on those instances without optimum solutions and those instances for which the best solution was unfeasible.

In [72]:
withoutOptimumOrUnfeasible = testSummary[(testSummary.optimumFO == -1) | (testSummary.bestFO < 0)]
withoutOptimumOrUnfeasible

,instanceName,optimumFO,bestFO,optimumRuntime,runtime,relativeError,n,c
27,n_600_c_10000000000_g_10_f_0.1_eps_1e-05_s_300,-1,9990699560,-1.000000,0.003270,9.990700e+09,600,10000000000
37,n_1000_c_10000000000_g_14_f_0.2_eps_0_s_300,-1,9998858493,-1.000000,0.005951,9.998858e+09,1000,10000000000
38,n_1200_c_10000000000_g_10_f_0.2_eps_0.0001_s_200,-1,9989496758,-1.000000,0.006778,9.989497e+09,1200,10000000000
40,n_1000_c_10000000000_g_14_f_0.2_eps_0.01_s_100,-1,9963834530,-1.000000,0.005264,9.963835e+09,1000,10000000000
46,n_800_c_10000000000_g_14_f_0.1_eps_0_s_200,-1,9998817315,-1.000000,0.006535,9.998817e+09,800,10000000000
61,n_800_c_10000000000_g_14_f_0.2_eps_0.0001_s_100,-1,9999722605,-1.000000,0.005403,9.999723e+09,800,10000000000
78,n_1000_c_10000000000_g_14_f_0.1_eps_0.01_s_200,-1,9944310866,-1.000000,0.004140,9.944311e+09,1000,10000000000
81,n_800_c_10000000000_g_10_f_0.3_eps_1e-05_s_200,-1,9991909164,-1.000000,0.004931,9.991909e+09,800,10000000000
106,n_400_c_10000000000_g_14_f_0.1_eps_0.01_s_300,-1,9934299105,-1.000000,0.002855,9.934299e+09,400,10000000000
113,n_1000_c_100000000_g_10_f_0.2_eps_0.01_s_100,100005870,-8050496004398263343,250.187495,0.007927,8.050023e+10,1000,100000000


# **Visualizations**

In [126]:
scatterPlotDF = testSummary[['instanceName', 'optimumRuntime', 'runtime', 'n', 'c']]
scatterPlotDF = scatterPlotDF.rename(columns = {'optimumRuntime': 'optimumRuntime(s)', 'runtime': 'runtime(s)'})
px.scatter(scatterPlotDF, x = 'runtime(s)', y = 'optimumRuntime(s)', color = 'c')